In [17]:
import pandas as pd
import pulp

df_s=pd.read_excel("/workspaces/Accident-Opt-Strabag/DataFiles/sources_hosp_toll.xlsx")
time_matrix_orsm=pd.read_csv("/workspaces/Accident-Opt-Strabag/DataFiles/Distance_Time_Matrix/osrm_time_matrix.csv") #for osrm based
time_matrix_chainage=pd.read_csv("/workspaces/Accident-Opt-Strabag/DataFiles/Distance_Time_Matrix/chainage_time_matrix.csv") #for chainage based
new_column_names = list(range(0, 16))
time_matrix_chainage.columns = new_column_names
time_matrix_orsm.columns = new_column_names

time_np_chainage=time_matrix_chainage.to_numpy()
time_np_orsm=time_matrix_orsm.to_numpy()

clusters=pd.read_csv("/workspaces/Accident-Opt-Strabag/DataFiles/KMeans_clusters.csv")
weights=clusters['Accident_Frequency'].to_list()

number_of_iterations=6

r1=[20,18,16,12,10,8]
r2= [30,25,20,16,12,10] 
alpha=[0.8, 0.9, 1.0]
alpha_values = [0.8, 0.9, 1.0]
time=[time_np_chainage,time_np_orsm]
count=1

for l in range(number_of_iterations):
        for t in range(len(time)):

            for a in alpha_values:

                time_np= time[t]
                if t==0:
                    t_matrix="chainage_time"
                else:
                    t_matrix="orsm_time"

                n=26 #number of demand points
                m=16 #potential sites ambulance location
                P=10 #number of amnbulances to be located
                alpha=a #reliability Taking 80% coverage within r1 time
                a_i = weights #weights at each site
                t_ij= time_np #shortedt time to travel from i to j  TIME MATRIX
                r_1= r1[l] #primary coverage time (mins)
                r_2= r2[l] #secondary coverage time (mins) ASSUMED 
                p_j=[] #maximum number of ambulances at each site IT WILL BE 1 for our case now
                for i in range(m):
                    p_j.append(2)

                
                print(r_1,r_2,a)
                print("--------------------")


                demand_points=[]
                for i in range(0,26):
                    demand_points.append(i)
                source_locations=[] #the potential locations of the ambulances 
                for i in range(0,16):
                    source_locations.append(i)

                # Define the new binary variables z_ij to represent a_ij2 * x_j
                z = pulp.LpVariable.dicts("z", [(i, j) for i in demand_points for j in source_locations], 0, 1, pulp.LpBinary)
                #new variable to represent a_ij1*n_j 
                x=pulp.LpVariable.dicts("x",[(i,j) for i in demand_points for j in source_locations], 0, 1, pulp.LpBinary)


                z_i1 = pulp.LpVariable.dicts("z_i1", demand_points, 0, 1, pulp.LpBinary)
                z_i2 = pulp.LpVariable.dicts("z_i2", demand_points, 0, 1, pulp.LpBinary)
                a_ij1 = pulp.LpVariable.dicts("a_ij1", [(i, j) for i in demand_points for j in source_locations], 0, 1, pulp.LpBinary)
                a_ij2 = pulp.LpVariable.dicts("a_ij2", [(i, j) for i in demand_points for j in source_locations], 0, 1, pulp.LpBinary)
                n_j = pulp.LpVariable.dicts("n_j", source_locations, 0, 2, pulp.LpInteger)

                problem=pulp.LpProblem("Ambulance-Coverage-Loc",sense=pulp.LpMaximize)
                problem+=pulp.lpSum(a_i[i]*z_i2[i] for i in demand_points)


                for i in demand_points:
                    for j in source_locations:
                        if t_ij[(i,j)]<=r_2 and n_j[j]!=0:
                            problem+= a_ij2[(i,j)]==1
                            problem+= z[(i,j)]==n_j[j]
                        else:
                            problem+= a_ij2[(i,j)]==0
                            problem+=z[(i,j)]==0
                        if t_ij[(i,j)]<=r_1 and n_j[j]!=0:
                            problem+=a_ij1[(i,j)]==1
                            problem+=x[(i,j)]==n_j[j]
                        else:
                            problem+=a_ij1[(i,j)]==0
                            problem+=x[(i,j)]==0

                #x for a_ij1*n_j z for a_ij2*n_j

                # Modify the original constraint to use the new binary variables
                for i in demand_points:
                    #problem += pulp.lpSum(a_ij2[(i, j)]*n_j for j in source_locations if t_ij[(i, j)] <= r_2) >= 1 #-------------------------------->2
                    problem += pulp.lpSum(z[(i, j)] for j in source_locations) >= 1


                # Constraints
                for i in demand_points:
                    problem += pulp.lpSum(a_i[i]*z_i1[i] for j in source_locations if t_ij[(i,j)]<=r_1)>=  alpha* pulp.lpSum(a_i[i]) #---------------------------->3
                    #problem += pulp.lpSum(x[(i,j)] for j in source_locations  if t_ij[(i, j)] <= r_1) >= z_i1[i] + z_i2[i] #------------------------>4
                    problem += pulp.lpSum(x[(i,j)] for j in source_locations) >= z_i1[i] + z_i2[i]
                    #problem += pulp.LpConstraint(z_i2[i]<=z_i1[i]) #------------------------------------>5

                for i in demand_points:
                    problem += z_i2[i]<=z_i1[i]

                for j in range(m):
                    #problem += pulp.LpConstraint(n_j[j] <= p_j[j]) #-------------------------->7
                    problem += n_j[j] <= p_j[j]

                for j in range(m):
                    #problem+=pulp.LpConstraint(n_j[j]>=0) #---------------------------------->8
                    problem+=n_j[j]>=0


                problem += pulp.lpSum(n_j[j] for j in source_locations ) == P #------------------------------>6

                problem.solve()

                # Solve the problem
                status = problem.solve()

                # Save solution outputs to a file
                with open(f"/workspaces/Accident-Opt-Strabag/DataFiles/Optimised_Allotments/opt_iteration_{l+1}_alpha_{a}_time_{t_matrix}.txt", "w") as file:
                    file.write("Solution status: {}\n\n".format(pulp.LpStatus[status]))
                    for j in source_locations:
                        file.write(f"Ambulance at location {j}: {n_j[j].varValue} \n")
                    
                    file.write("\n")

                    for k in demand_points:
                        file.write(f"Demand point {k} primary coverage (z_i1): {z_i1[k].varValue}\n")
                        file.write(f"Demand point {k} secondary coverage (z_i2): {z_i2[k].varValue}\n")

                    file.write("\n")
                    file.write(f"Objective function value: {pulp.value(problem.objective)}\n \n")

                    file.write(f"r_1: {r_1} |\t")
                    file.write(f"r_2: {r_2} |\t")
                    file.write(f"alpha: {alpha} |\t")


        
                allotment_list=[]
                for n in source_locations:
                    allotment_list.append(int(n_j[n].varValue))

                df=pd.DataFrame({'optimised-ambulances':allotment_list,'lat':df_s['Source_Latitude'],'long':df_s['Source_Longitude']})
                df.to_csv(f"/workspaces/Accident-Opt-Strabag/DataFiles/Optimised_Allotments/opt_Iteration_{l+1}_alpha_{a}_time_{t_matrix}.csv") 

                print("Loop Count:",count)
                print("------------------")
                count+=1  

20 30 0.8
--------------------
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /workspaces/Accident-Opt-Strabag/venv-efkon/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c2c2a7bc7b8d4e51a718e3abef657f25-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/c2c2a7bc7b8d4e51a718e3abef657f25-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1806 COLUMNS
At line 8462 RHS
At line 10264 BOUNDS
At line 11997 ENDATA
Problem MODEL has 1801 rows, 1732 columns and 3165 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 107 - 0.00 seconds
Cgl0002I 682 variables fixed
Cgl0004I processed model has 49 rows, 42 columns (42 integer (42 of which binary)) and 489 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -107
Cbc0038I Before